# Modelling metabolism in SOY

### Generating SOY core model

In [1]:
import pandas as pd
from cobra import io
from cobra.core import Metabolite, Reaction
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "C:\\Users\\FFAR\\Downloads\\plantcoremetabolism-model\\PlantCoreMetabolism_v1_3_2.xml"
model = io.read_sbml_model(fname)
model.solver='glpk'
display.clear_output()

from sweetlovegroup import transform
model = transform.fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
met.compartment = "b"
rxn.add_metabolites({met:1})


for rxn in model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_bound = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000


model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_m").remove_from_model()
model.metabolites.get_by_id("aL_1_PHOSPHATIDYL_GLYCEROL_P_p").remove_from_model()

model.reactions.get_by_id("PGPPHOSPHA_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_m"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_m").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_m"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_m"):0.03})
model.reactions.get_by_id("PGPPHOSPHA_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03,
                                                               model.metabolites.get_by_id("PROTON_p"):-0.03})
model.reactions.get_by_id("PHOSPHAGLYPSYN_RXN_p").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):0.03,
                                                                   model.metabolites.get_by_id("PROTON_p"):0.03})
model.reactions.get_by_id("LPG_biosynthesis_c").add_metabolites({model.metabolites.get_by_id("L_1_PHOSPHATIDYL_GLYCEROL_P_p"):-0.03})

In [2]:
import pandas as pd

df = pd.read_csv("Data/biomass_soy.csv")

FA=["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p","OLEATE_CPD_p",
    "Octadecadienoate_p","LINOLENIC_ACID_p","ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]
FACP = {"PALMITATE_p":"Palmitoyl_ACPs_p",
        "CPD_9245_p":"Palmitoleoyl_ACP_p",
        "CPD_17412_p":"hexadecadienoate_ACP_p",
        "CPD_17291_p":"hexadecatrienoate_ACP_p",
        "STEARIC_ACID_p":"Stearoyl_ACPs_p",
        "OLEATE_CPD_p":"Oleoyl_ACPs_p",
        "Octadecadienoate_p":"Octadecadienoyl_ACP_p",
        "LINOLENIC_ACID_p":"Octadecatrienoyl_ACP_p",
        "ARACHIDIC_ACID_p":"Arachidoyl_ACPs_p",
        "CPD_16709_p":"Eicosenoyl_ACP_p",
        "DOCOSANOATE_p":"Behenoyl_ACPs_p"}


PLs = ["ACYL_SN_GLYCEROL_3P_p",
       "L_PHOSPHATIDATE_p","L_PHOSPHATIDATE_m","DIACYLGLYCEROL_p",
       "DIACYLGLYCEROL_r","Triacylglycerols_p","PHOSPHATIDYL_CHOLINE_r",
       "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","L_1_PHOSPHATIDYL_GLYCEROL_p",
       "L_1_PHOSPHATIDYL_GLYCEROL_P_p","L_1_PHOSPHATIDYL_GLYCEROL_P_m",
       "L_1_PHOSPHATIDYL_GLYCEROL_m","2_Lysophosphatidylcholines_r",
       "Lysophosphatidylglycerols_r","CDPDIACYLGLYCEROL_p","CDPDIACYLGLYCEROL_m",
       "D_Galactosyl_12_diacyl_glycerols_p","Galactosyl_galactosyl_diacyl_glycerols_p"]


for met in PLs:
    met=model.metabolites.get_by_id(met)
    met.formula=""

### Use biomass composition to generate biomass equations

In [3]:
def generateMissingFormula(model,debug=False):
    loop_counter = 0
    former = 0
    for met in model.metabolites:
        if met.formula == "" or met.formula == "NA":
            former = former +1
    latter = 1
    while True:
        loop_counter = loop_counter+1
        if debug:
            print("Loop = "+str(loop_counter))
        former = latter
        for rxn in model.reactions:
            count = 0
            for met in rxn.metabolites:
                if met.formula=="" or met.formula=="NA" or met.formula == None:
                    if met.formula == "NA" or met.formula == None:
                        met.formula = ""
                    count = count + 1
                    coeff = rxn.metabolites[met]
            if count == 1:
                unb = rxn.check_mass_balance()
                eqn = rxn.reaction
                eqn = " "+eqn+" "
                for met in rxn.metabolites.keys():
                    formula = met.formula
                    if formula == None:
                        formula = "0"
                        NF_list.add(rxn.id)
                    eqn=eqn.replace(" "+met.id+" ","("+formula+")")
                if debug:
                    print(eqn)
                    print(unb)
                for met in rxn.metabolites:
                    if met.formula == "":
                        tempForm = ""
                        for a in sorted(unb.keys()):
                            if a=="charge" or round(unb[a],2)==0:
                                continue
                            num = float(abs(unb[a]))/abs(coeff)
                            if str(round(num))==str(num):
                                tempForm = tempForm+a+str(int(round(num)))
                            else:
                                tempForm = tempForm+a+str(num)
                                #print(a)
                                #print(round(num)==num)
                                #print(round(num))
                                #print(num)
                                #print(type(round(num)))
                                #print(type(num))
                        met.formula = tempForm
                        if debug:
                            print(met.id)
                            print(tempForm)
        latter = 0
        for met in model.metabolites:
            if met.formula == "" or met.formula == "NA":
                latter = latter +1
        if former == latter:
            break
        
        

### Run stem

In [4]:
temp  = model.copy()

for met in temp.reactions.Phloem_output_tx.metabolites.keys():
    met2 = met.copy()
    if met.id=="sSUCROSE_b":
        met2.id = met.id.replace("_b","_ph")
    elif "PROTON" in met.id:
        continue
    else:
        met2.id = met.id.replace("_c","_ph")
    met2.compartment = "ph"
    temp.add_metabolites(met2)
    
    rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
    rxn.add_metabolites({met2:-1,temp.metabolites.get_by_id("PROTON_e"):-1,met:1,temp.metabolites.get_by_id("PROTON_c"):1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    temp.add_reaction(rxn)
    
    print(rxn.reaction)
    

GLN_ph + PROTON_e --> GLN_c + PROTON_c
ASN_ph + PROTON_e --> ASN_c + PROTON_c
ILE_ph + PROTON_e --> ILE_c + PROTON_c
L_ALPHA_ALANINE_ph + PROTON_e --> L_ALPHA_ALANINE_c + PROTON_c
GLT_ph + PROTON_e --> GLT_c + PROTON_c
PROTON_e + VAL_ph --> PROTON_c + VAL_c
GLY_ph + PROTON_e --> GLY_c + PROTON_c
PROTON_e + THR_ph --> PROTON_c + THR_c
4_AMINO_BUTYRATE_ph + PROTON_e --> 4_AMINO_BUTYRATE_c + PROTON_c
LYS_ph + PROTON_e --> LYS_c + PROTON_c
PROTON_e + SER_ph --> PROTON_c + SER_c
LEU_ph + PROTON_e --> LEU_c + PROTON_c
PROTON_e + sSUCROSE_ph --> PROTON_c + sSUCROSE_b
L_ASPARTATE_ph + PROTON_e --> L_ASPARTATE_c + PROTON_c
GLC_ph + PROTON_e --> GLC_c + PROTON_c
FRU_ph + PROTON_e --> FRU_c + PROTON_c
PHE_ph + PROTON_e --> PHE_c + PROTON_c
ARG_ph + PROTON_e --> ARG_c + PROTON_c
HIS_ph + PROTON_e --> HIS_c + PROTON_c
CYS_ph + PROTON_e --> CYS_c + PROTON_c
PROTON_e + PRO_ph --> PROTON_c + PRO_c
MET_ph + PROTON_e --> MET_c + PROTON_c
PROTON_e + TYR_ph --> PROTON_c + TYR_c
PROTON_e + TRP_ph --> PROTO

In [5]:
def generateStemModel(model):
    for met in model.reactions.Phloem_output_tx.metabolites.keys():
        met2 = met.copy()
        if met.id=="sSUCROSE_b":
            met2.id = "SUCROSE_ph"
            met = model.metabolites.get_by_id("SUCROSE_c")
        elif "PROTON" in met.id:
            continue
        else:
            met2.id = met.id.replace("_c","_ph")
        met2.compartment = "ph"
        model.add_metabolites(met2)
        
        rxn = Reaction(met2.id+"_exchange")
        rxn.add_metabolites({met2:1})
        model.add_reaction(rxn)

        rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
        rxn.add_metabolites({met2:-1,model.metabolites.get_by_id("PROTON_e"):-1,
                             met:1,model.metabolites.get_by_id("PROTON_c"):1})
        rxn.lower_bound = 0
        rxn.upper_bound = 1000
        model.add_reaction(rxn)

        #print(rxn.reaction)
    return model

In [6]:
from cobra import flux_analysis
stem_model = model.copy()

k = "stem"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({stem_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({stem_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({stem_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({stem_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 1000
stem_model.add_reaction(RXN1)

RXN2.add_metabolites({stem_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 1000
stem_model.add_reaction(RXN2)

generateMissingFormula(stem_model)

rxn = Reaction("Biomass_stem_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    else:
        rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
stem_model.add_reaction(rxn)

stem_model = generateStemModel(stem_model)

stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 0
stem_model.reactions.Sucrose_tx.lower_bound = 0
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1

flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

cobra\core\metabolite.py:104 UserWarning: 17.612021999304808 is not an integer (in formula C17.612021999304808H32.696359514628455O0.27067590171015216X1.0)
cobra\core\metabolite.py:104 UserWarning: 32.696359514628455 is not an integer (in formula C17.612021999304808H32.696359514628455O0.27067590171015216X1.0)
cobra\core\metabolite.py:104 UserWarning: 0.27067590171015216 is not an integer (in formula C17.612021999304808H32.696359514628455O0.27067590171015216X1.0)
cobra\core\metabolite.py:104 UserWarning: 17.612021999304808 is not an integer (in formula C17.612021999304808H33.180142488275095O2.0)
cobra\core\metabolite.py:104 UserWarning: 33.180142488275095 is not an integer (in formula C17.612021999304808H33.180142488275095O2.0)
cobra\core\metabolite.py:104 UserWarning: 20.612021999304808 is not an integer (in formula C20.612021999304808H38.696359514628455O6.2706759017101525P1.0)
cobra\core\metabolite.py:104 UserWarning: 38.696359514628455 is not an integer (in formula C20.612021999304808

Biomass flux =39.89663607922134


In [7]:
stem_model.reactions.Biomass_stem_tx.reaction

'7.659408551696633e-05 CYS_c + 0.6624127272448725 Cellulose_b + 0.0008781839371548684 FRU_c + 0.00010707394063407757 GLC_c + 1.838367529633685e-05 GLY_c + 8.538347701644519e-06 ILE_c + 2.1193398759439072e-05 LEU_c + 9.782907765522723e-06 LYS_c + 5.675545815562533e-06 L_PHOSPHATIDATE_p + 7.975265150758646e-06 MET_c + 2.558895107947629e-05 MYO_INOSITOL_c + 5.93259338961387e-06 PHE_c + 6.861780974383322e-06 PRO_c + 1.1975309152513402e-05 SUC_c + 2.7407527792043203 Starch_b + 7.463115085767901e-05 THR_c + 4.514634314068649e-05 TYR_c + 9.816781301063267e-06 VAL_c + 12.522273984514358 Xylan_b + 4.045921323046277 pALA_b + 0.004964837109152188 pASN_b + 3.1123548220012403 pASP_b + 0.1476511846744479 pCYS_b + 0.17311367516396445 pGLN_b + 2.2107066580017207 pGLU_b + 4.183739547282015 pGLY_b + 2.0708777474801967 pILE_b + 2.893926596282221 pLEU_b + 0.5821794959764395 pLYS_b + 0.5362542739043828 pMET_b + 1.6533728379480637 pPHE_b + 3.042380060845757 pPRO_b + 2.7238288507142943 pSER_b + 1.89802983248

### Run root

In [8]:
def generateRootModel(model):
    for met in model.reactions.Phloem_output_tx.metabolites.keys():
        met2 = met.copy()
        if met.id=="sSUCROSE_b":
            met2.id = "SUCROSE_ph"
            met = model.metabolites.get_by_id("SUCROSE_c")
        elif "PROTON" in met.id:
            continue
        else:
            met2.id = met.id.replace("_c","_ph")
        met2.compartment = "ph"
        model.add_metabolites(met2)
        
        rxn = Reaction(met2.id+"_exchange")
        rxn.add_metabolites({met2:1})
        model.add_reaction(rxn)

        rxn = Reaction(met2.id.replace("_ph","_phloem_uptake"),name=met2.id.replace("_ph","_phloem_uptake"))
        rxn.add_metabolites({met2:-1,model.metabolites.get_by_id("PROTON_e"):-1,
                             met:1,model.metabolites.get_by_id("PROTON_c"):1})
        rxn.lower_bound = 0
        rxn.upper_bound = 1000
        model.add_reaction(rxn)

    #add xylem reactions
    for met in ["CAII","MGII","KI","NITRATE","SULFATE","AMMONIUM","WATER","GLT","L_ASPARTATE","ASN","GLN"]:
        met2 = model.metabolites.get_by_id(met+"_c").copy()
        met2.id = met+"_xy"
        met2.compartment = "xy"

        rxn = Reaction(met+"_exchange")
        rxn.add_metabolites({met2:-1})
        rxn.lower_bound = 0
        rxn.upper_bound = 1000
        model.add_reaction(rxn)

        rxn = Reaction(met+"_xylem_export")
        rxn.add_metabolites({model.metabolites.get_by_id(met+"_c"):-1,met2:1})
        model.add_reaction(rxn)
        
    #adding symbiont compartment
    from cobra import io
    rhizo = io.read_sbml_model("Data/iGD1348.xml")
    rhizo.compartments={"c0":"rhizobe cytosol","e0":"rhizobe extracellular"}
    model = model+rhizo
    
    rxn = Reaction("Sucrose_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("SUCROSE_c"):-1,model.metabolites.get_by_id("cpd00076[e0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    rxn = Reaction("Alanine_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("L_ALPHA_ALANINE_c"):-1,model.metabolites.get_by_id("cpd00035[e0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    rxn = Reaction("Glutamate_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("GLT_c"):-1,model.metabolites.get_by_id("cpd00023[e0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    rxn = Reaction("Malate_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("MAL_c"):-1,model.metabolites.get_by_id("cpd00130[e0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    rxn = Reaction("Succinate_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("SUC_c"):-1,model.metabolites.get_by_id("cpd00036[e0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    rxn = Reaction("Ammonium_exchange_symbiont")
    rxn.name = rxn.id.replace("_"," ")
    rxn.add_metabolites({model.metabolites.get_by_id("AMMONIUM_c"):-1,model.metabolites.get_by_id("cpdFixed[c0]"):1})
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000
    model.add_reaction(rxn)
    
    return model

In [9]:
from cobra import flux_analysis
root_model = model.copy()

k = "root"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({root_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({root_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({root_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({root_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 1000
root_model.add_reaction(RXN1)
    
RXN2.add_metabolites({root_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 1000
root_model.add_reaction(RXN2)


generateMissingFormula(root_model)

rxn = Reaction("Biomass_root_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)

root_model = generateRootModel(root_model)

root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 0
root_model.reactions.Sucrose_tx.lower_bound = 0
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0


root_model.reactions.Nitrate_tx.upper_bound = 0
root_model.reactions.Nitrate_tx.lower_bound = 0
root_model.reactions.NH4_tx.upper_bound = 0
root_model.reactions.NH4_tx.lower_bound = 0
#root_model.reactions.EX_cpdFixed_e0.upper_bound = 0
#root_model.reactions.EX_cpdFixed_e0.lower_bound = 0
#root_model.reactions.EX_cpd00013_e0.lower_bound = 0
#root_model.reactions.EX_cpd00013_e0.upper_bound = 0 

for rxn in root_model.reactions.query("EX_cpd"):
    if rxn.id in [#"EX_cpd00076_e0",
                  "EX_cpd00001_e0","EX_cpd00067_e0","EX_cpd00011_e0",
                  "EX_cpd00007_e0","EX_cpd00149_e0","EX_cpd00009_e0",
                  "EX_cpd00254_e0","EX_cpd00063_e0","EX_cpd00030_e0",
                  "EX_cpd10515_e0","EX_cpd10516_e0","EX_cpd00034_e0",
                  "EX_cpd00205_e0","EX_cpd00971_e0","EX_cpd00104_e0",
                  "EX_cpd00048_e0","EX_cpd11416_e0","EX_cpd00098_e0",
                  "EX_cpd00528_e0"]:#"EX_cpd00182_e0","EX_cpd00311_e0","EX_cpd00064_e0",
        continue
    rxn.lower_bound = 0
    rxn.upper_bound = 0
    
for rxn in root_model.reactions.query("_phloem_"):
    if "SUCROSE" not in rxn.id :
        rxn.lower_bound = 0
        rxn.upper_bound = 0
    else:
        rxn.upper_bound = 10
        

root_model.reactions.EX_cpd00013_e0.lower_bound = 0
root_model.reactions.EX_cpd00013_e0.upper_bound = 0
root_model.reactions.EX_cpdFixed_e0.upper_bound = 0
root_model.reactions.EX_cpdFixed_e0.lower_bound = 0

root_model.reactions.GLN_exchange.lower_bound = 0
for rxn in root_model.reactions:
    rxn.objective_coefficient=0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux = "+str(root_model.reactions.Biomass_root_tx.flux))

cobra\core\metabolite.py:104 UserWarning: 17.700746860586182 is not an integer (in formula C17.700746860586182H33.000213348999786O0.3450481094278798X1.0)
cobra\core\metabolite.py:104 UserWarning: 33.000213348999786 is not an integer (in formula C17.700746860586182H33.000213348999786O0.3450481094278798X1.0)
cobra\core\metabolite.py:104 UserWarning: 0.3450481094278798 is not an integer (in formula C17.700746860586182H33.000213348999786O0.3450481094278798X1.0)
cobra\core\metabolite.py:104 UserWarning: 17.700746860586182 is not an integer (in formula C17.700746860586182H33.30951059910088O2.0)
cobra\core\metabolite.py:104 UserWarning: 33.30951059910088 is not an integer (in formula C17.700746860586182H33.30951059910088O2.0)
cobra\core\metabolite.py:104 UserWarning: 20.700746860586182 is not an integer (in formula C20.700746860586182H39.000213348999786O6.3450481094278794P1.0)
cobra\core\metabolite.py:104 UserWarning: 39.000213348999786 is not an integer (in formula C20.700746860586182H39.000

Biomass flux = 0.0


In [10]:
met = root_model.metabolites.get_by_id("cpd00013[c0]")
for rxn in met.reactions:
    if round(rxn.flux,7)!=0:
        eqn = rxn.reaction
        for met2 in rxn.metabolites.keys():
            eqn = eqn.replace(met2.id,met2.name)
        #print(rxn.id+"\t"+str(rxn.metabolites[met]*rxn.flux))
        print(rxn.id+"\t"+str(rxn.metabolites[met]*rxn.flux)+"\t"+eqn)
        #print(rxn.metabolites)
        #print(eqn+"\t"+str(rxn.metabolites[met]))
        #print(eqn+"\t"+str(rxn.flux))
        #print("------------------")

In [11]:
for rxn in root_model.metabolites.get_by_id("cpd00008[c0]").reactions:
    skip = False
    eqn = rxn.reaction
    for met2 in rxn.metabolites.keys():
        if "ATP" in met2.name:
            skip = True
        eqn = eqn.replace(met2.id,met2.name)
    if skip==True:
        continue
    print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.lower_bound)+"\t"+str(rxn.upper_bound))
    print(eqn)
    print("----------")

rxn08615	cpd00387[c0] + cpd15302[c0] <=> cpd00008[c0] + cpd00067[c0] + cpd00155[c0]	-1000.0	1000.0
ADPglucose_c0 + glycogenn_1_c0 <=> ADP_c0 + H_c0 + Glycogen_c0
----------
rxn10052	cpd00018[c0] + cpd00067[c0] + cpd00421[c0] <=> cpd00008[c0] + cpd00012[c0]	-1000.0	1000.0
AMP_c0 + H_c0 + Triphosphate_c0 <=> ADP_c0 + PPi_c0
----------
rxn05231	cpd00008[c0] + cpd00067[c0] + cpd11421[c0] --> cpd00001[c0] + cpd00177[c0] + cpd11420[c0]	0.0	1000.0
ADP_c0 + H_c0 + trdrd_c0 --> H2O_c0 + dADP_c0 + trdox_c0
----------


In [24]:
temp = root_model.copy()
#temp.metabolites.get_by_id("cpd11620[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd11621[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00002[c0]").remove_from_model()

#temp.metabolites.get_by_id("cpd00008[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00009[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00067[e0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00067[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00001[c0]").remove_from_model()
#temp.metabolites.get_by_id("cpd00018[c0]").remove_from_model()

rxn = Reaction("temp1")
rxn.add_metabolites({temp.metabolites.get_by_id("cpdFixed[c0]"):-1,
                     temp.metabolites.get_by_id("cpd00013[c0]"):1})
#rxn.add_metabolites({temp.metabolites.get_by_id("cpd00013[c0]"):-1})
rxn.upper_bound = 1000
rxn.lower_bound = -1000
temp.add_reaction(rxn)

rxn = Reaction("temp2")
rxn.add_metabolites({#temp.metabolites.get_by_id("cpd00001[c0]"):-16,
                     #temp.metabolites.get_by_id("cpd00002[c0]"):-16,
                     temp.metabolites.get_by_id("cpd00528[c0]"):-1,
                     #temp.metabolites.get_by_id("cpd11620[c0]"):-4,
                     #temp.metabolites.get_by_id("cpd00008[c0]"):16,
                     #temp.metabolites.get_by_id("cpd00009[c0]"):16,
                     temp.metabolites.get_by_id("cpd00067[c0]"):-6,
                     #temp.metabolites.get_by_id("cpd11621[c0]"):4,
                     temp.metabolites.get_by_id("cpd11640[c0]"):1,
                     temp.metabolites.get_by_id("cpdFixed[c0]"):2})
# # rxn.add_metabolites({temp.metabolites.get_by_id("cpd00018[c0]"):-1})
# rxn.lower_bound = -1000
temp.add_reaction(rxn)

eqn = rxn.reaction
for met2 in rxn.metabolites.keys():
    eqn = eqn.replace(met2.id,met2.name)
print(eqn)

flux_analysis.parsimonious.pfba(temp)
print("Biomass flux ="+str(temp.reactions.Biomass_root_tx.flux))

6 H_c0 + N2_c0 --> H2_c0 + 2 Fixed ammonium_c0
Biomass flux =0.0


In [23]:
met = temp.metabolites.get_by_id("cpdFixed[c0]")
for rxn in met.reactions:
#     if round(rxn.flux,7)!=0:
        eqn = rxn.reaction
        for met2 in rxn.metabolites.keys():
            eqn = eqn.replace(met2.id,met2.name)
        #print(rxn.id+"\t"+str(rxn.metabolites[met]*rxn.flux))
        print(rxn.id+"\t"+str(rxn.metabolites[met]*rxn.flux)+"\t"+eqn)
        print(rxn.reaction)
        #print(eqn+"\t"+str(rxn.metabolites[met]))
        #print(eqn+"\t"+str(rxn.flux))
        #print("------------------")

symbiosis_c0	-0.0	L_Alanine_c0 + L_Aspartate_c0 + L_Arginine_c0 + 3.0 H_c0 + Glycogen_c0 + Palmitate_c0 + Fixed ammonium_c0 + 0.01 Symbiosis cofactors_c0 --> L_Alanine_e0 + L_Aspartate_e0 + glycogenn_1_c0 + Fixed ammonium_e0
cpd00035[c0] + cpd00041[c0] + cpd00051[c0] + 3.0 cpd00067[c0] + cpd00155[c0] + cpd00214[c0] + cpdFixed[c0] + 0.01 cpdSymCoF[c0] --> cpd00035[e0] + cpd00041[e0] + cpd15302[c0] + cpdFixed[e0]
temp2	0.0	6 H_c0 + N2_c0 --> H2_c0 + 2 Fixed ammonium_c0
6 cpd00067[c0] + cpd00528[c0] --> cpd11640[c0] + 2 cpdFixed[c0]
rxn06874_c0	0.0	16.0 H2O_c0 + 16.0 ATP_c0 + N2_c0 + 4.0 Reducedferredoxin_c0 --> 16.0 ADP_c0 + 16.0 Phosphate_c0 + 6.0 H_c0 + 4.0 Oxidizedferredoxin_c0 + H2_c0 + 2.0 Fixed ammonium_c0
16.0 cpd00001[c0] + 16.0 cpd00002[c0] + cpd00528[c0] + 4.0 cpd11620[c0] --> 16.0 cpd00008[c0] + 16.0 cpd00009[c0] + 6.0 cpd00067[c0] + 4.0 cpd11621[c0] + cpd11640[c0] + 2.0 cpdFixed[c0]
Ammonium_exchange_symbiont	0.0	AMMONIUM <=> Fixed ammonium_c0
AMMONIUM_c <=> cpdFixed[c0]


In [72]:
for rxn in temp.reactions.query("exchange"):
    if round(rxn.flux,7)!=0:
        eqn = rxn.reaction
        for met2 in rxn.metabolites.keys():
            eqn = eqn.replace(met2.id,met2.name)
        #print(rxn.id+"\t"+str(rxn.metabolites[met]*rxn.flux))
        print(rxn.id+"\t"+str(rxn.flux)+"\t"+eqn)
        print(rxn.reaction)
        #print(eqn+"\t"+str(rxn.metabolites[met]))
        #print(eqn+"\t"+str(rxn.flux))
        #print("------------------")

SUCROSE_ph_exchange	0.7381768150238426	 --> sSUCROSE[b]
 --> SUCROSE_ph
Sucrose_exchange_symbiont	0.24999999999999398	SUCROSE <=> Sucrose_e0
SUCROSE_c <=> cpd00076[e0]
Alanine_exchange_symbiont	-0.9999999999999749	L-ALPHA-ALANINE <=> L_Alanine_e0
L_ALPHA_ALANINE_c <=> cpd00035[e0]


In [64]:
for rxn in temp.reactions:
    if round(rxn.flux,7)==0:
        continue
    if len(rxn.reactants)==0 or len(rxn.products)==0:
        eqn = rxn.reaction
        for met2 in rxn.metabolites.keys():
            eqn = eqn.replace(met2.id,met2.name)
        print(rxn.id+"\t"+eqn+"\t"+str(rxn.flux))

H_tx	PROTON --> 	0.06439159115954657
H2O_tx	 <=> WATER	-2.1479356960515594
CO2_tx	 <=> CARBON-DIOXIDE	-1.5122562605512555
O2_tx	 <=> OXYGEN-MOLECULE	1.2258448279988976
Pi_tx	 <=> Pi[e]	1.3045641055512395e-07
SO4_tx	 <=> SULFATE	0.029312325460245696
Biomass_root_tx	5.8518541628802956e-05 CYS + 0.6480437957395563 Cellulose[b] + 0.0009546732529629342 FRU[c] + 5.117790215895257e-05 GLC + 8.658977494651417e-06 GLY + 1.1282816605744545e-05 ILE + 2.8511982503705806e-05 LEU + 9.375286608625942e-06 LYS + 3.5567809884531068e-06 L-PHOSPHATIDATE + 4.825370511383384e-06 MET + 4.16306145544625e-05 MYO-INOSITOL + 5.569373386168123e-06 PHE + 1.3984135909819174e-05 PRO + 8.356870415782735e-06 SUC + 1.827811330942959 Starch[b] + 5.674989649020359e-05 THR + 2.9858400536810998e-05 TYR + 1.7499479710591047e-05 VAL + 13.015276386519526 Xylan[b] + 3.039393987283876 pALA[b] + 6.898323860987215e-05 pASN[b] + 2.2788993755542424 pASP[b] + 0.11753326040387865 pCYS[b] + 0.22272671636633606 pGLN[b] + 1.771430237558

In [60]:
for met in temp.metabolites:
    met.charge = int(met.charge)
io.write_sbml_model(temp,"tempSBML.xml")

In [58]:
dir(io)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'absolute_import',
 'dict',
 'from_json',
 'from_yaml',
 'json',
 'load_json_model',
 'load_matlab_model',
 'load_yaml_model',
 'mat',
 'model_from_dict',
 'model_to_dict',
 'read_sbml_model',
 'save_json_model',
 'save_matlab_model',
 'save_yaml_model',
 'sbml',
 'to_json',
 'to_yaml',
 'validate_sbml_model',
 'write_sbml_model',
 'yaml']

### Run seed

In [ ]:
from cobra import flux_analysis
seed_model = model.copy()

k = "seed"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({seed_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({seed_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({seed_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({seed_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 1000
seed_model.add_reaction(RXN1)
    
RXN2.add_metabolites({seed_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 1000
seed_model.add_reaction(RXN2)


generateMissingFormula(seed_model)

rxn = Reaction("Biomass_seed_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 1
seed_model.reactions.Sucrose_tx.lower_bound = 1
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

In [ ]:
seed_model.reactions.Biomass_seed_tx.reaction

### Generate diel leaf model



In [ ]:
from cobra import flux_analysis
leaf_model = model.copy()

k = "leaf"
RXN1 = Reaction("Fatty_acid_mix_"+k)
RXN2 = Reaction("Fatty_acid_ACP_"+k)
tot = 0
for met in df["Unnamed: 0"]:
    #print met
    if met in FA:
        RXN1.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
        RXN2.add_metabolites({leaf_model.metabolites.get_by_id(FACP[met]):-1*float(df[df["Unnamed: 0"]==met][k])})
        tot = tot+(float(df[df["Unnamed: 0"]==met][k]))
if tot==0:
    RXN1.add_metabolites({leaf_model.metabolites.PALMITATE_p:-1})
    RXN2.add_metabolites({leaf_model.metabolites.Palmitoyl_ACPs_p:-1})
    tot = 1
RXN1.add_metabolites({leaf_model.metabolites.Fatty_Acids_p:tot})
RXN1.lower_bound = 0
RXN1.upper_bound = 1000
leaf_model.add_reaction(RXN1)
    
RXN2.add_metabolites({leaf_model.metabolites.Fatty_acyl_ACP_p:tot})
RXN2.lower_bound = 0
RXN2.upper_bound = 1000
leaf_model.add_reaction(RXN2)


generateMissingFormula(leaf_model)

rxn = Reaction("Biomass_leaf_tx")
for met in df["Unnamed: 0"]:
    if met in FA:
        continue
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})

rxn.lower_bound = 0
rxn.upper_bound = 1000
leaf_model.add_reaction(rxn)

leaf_model.reactions.GLC_tx.upper_bound = 0
leaf_model.reactions.GLC_tx.lower_bound = 0
leaf_model.reactions.Sucrose_tx.upper_bound = 1
leaf_model.reactions.Sucrose_tx.lower_bound = 1
leaf_model.reactions.Photon_tx.upper_bound = 0
leaf_model.reactions.Photon_tx.lower_bound = 0
leaf_model.reactions.Biomass_leaf_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(leaf_model)
print("Biomass flux ="+str(leaf_model.reactions.Biomass_leaf_tx.flux))

    
    

In [ ]:
from sweetlovegroup.transform import setupC3DielModel
leaf_model = setupC3DielModel(leaf_model)

In [ ]:
leaf_model.reactions.Phloem_output_tx1.objective_coefficient = 0
leaf_model.reactions.Phloem_output_tx2.objective_coefficient = 0

leaf_model.reactions.Photon_tx1.upper_bound = 500
leaf_model.reactions.Photon_tx1.lower_bound = 500

leaf_model.reactions.Biomass_leaf_tx1.objective_coefficient = 0

from cobra.flux_analysis import pfba
sol = pfba(leaf_model)
print("Phloem export rate ="+str(sol.fluxes["diel_biomass"]*4))

In [ ]:
growing_leaf_model = leaf_model.copy()
met1=Metabolite("Leaf_biomass1")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx1
rxn.add_metabolites({met1:1})

met2=Metabolite("Leaf_biomass2")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx2
rxn.add_metabolites({met2:1})

growing_leaf_model.reactions.diel_biomass.add_metabolites({growing_leaf_model.metabolites.X_Phloem_contribution_t1:3,
                                                           growing_leaf_model.metabolites.X_Phloem_contribution_t2:1,
                                                           met1:-3, met2:-1})

from cobra.flux_analysis import pfba
sol = pfba(growing_leaf_model)
print("Phloem export rate ="+str(sol.fluxes["diel_biomass"]*4))